# TRAINING **CODE**

In [ ]:

import os
import torch
import cv2
import numpy as np
import random
from google.colab.patches import cv2_imshow

# Install dependencies
!pip install pyyaml==5.1
!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'

from detectron2.utils.logger import setup_logger
setup_logger()

# Import libraries
from detectron2 import model_zoo
from detectron2.engine import DefaultTrainer, DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.data.datasets import register_coco_instances
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader

# Define dataset paths
SAVE_PATH = "/content/drive/MyDrive/Dataset/SafetyEquipment"
os.makedirs(SAVE_PATH, exist_ok=True)

# Download dataset
!curl -L "https://universe.roboflow.com/ds/CmwBTV2FCH?key=wfRQPKfam9" > safety_dataset.zip
!unzip safety_dataset.zip -d {SAVE_PATH}
!rm safety_dataset.zip

# Register dataset
DATASET_PATH = f"{SAVE_PATH}/train/_annotations.coco.json"
TEST_PATH = f"{SAVE_PATH}/valid/_annotations.coco.json"

register_coco_instances("safety_train", {}, DATASET_PATH, f"{SAVE_PATH}/train")
register_coco_instances("safety_valid", {}, TEST_PATH, f"{SAVE_PATH}/valid")

# Load metadata
train_metadata = MetadataCatalog.get("safety_train")
dataset_dicts = DatasetCatalog.get("safety_train")

# Visualize sample images
for d in random.sample(dataset_dicts, 3):
    img = cv2.imread(d["file_name"])
    visualizer = Visualizer(img[:, :, ::-1], metadata=train_metadata, scale=0.5)
    vis = visualizer.draw_dataset_dict(d)
    cv2_imshow(vis.get_image()[:, :, ::-1])

# Configure model
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("safety_train",)
cfg.DATASETS.TEST = ("safety_valid",)
cfg.DATALOADER.NUM_WORKERS = 4
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml")
cfg.SOLVER.IMS_PER_BATCH = 4
cfg.SOLVER.BASE_LR = 0.001
cfg.SOLVER.MAX_ITER = 5000
cfg.SOLVER.STEPS = [3000, 4500]
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 256
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 25  # Adjust based on dataset
cfg.OUTPUT_DIR = SAVE_PATH

# Train model
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=True)
trainer.train()

# Save configuration
with open(f"{SAVE_PATH}/config.yml", "w") as f:
    f.write(cfg.dump())

# Evaluate model
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.6
predictor = DefaultPredictor(cfg)

evaluator = COCOEvaluator("safety_valid", output_dir=SAVE_PATH)
val_loader = build_detection_test_loader(cfg, "safety_valid")
print(inference_on_dataset(predictor.model, val_loader, evaluator))

# Visualize test results
for imageName in random.sample(os.listdir(f"{SAVE_PATH}/valid"), 5):
    im = cv2.imread(os.path.join(f"{SAVE_PATH}/valid", imageName))
    outputs = predictor(im)
    v = Visualizer(im[:, :, ::-1], metadata=train_metadata, scale=0.8)
    out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    cv2_imshow(out.get_image()[:, :, ::-1])

In [ ]:
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader

# Define evaluator
evaluator = COCOEvaluator("safety_valid", output_dir=SAVE_PATH)
val_loader = build_detection_test_loader(cfg, "safety_valid")

# Run evaluation
print(inference_on_dataset(predictor.model, val_loader, evaluator))


In [ ]:
# Step 1: Load Dependencies
import torch
import cv2
import matplotlib.pyplot as plt
from google.colab import files
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog

# Step 2: Load Trained Model
cfg = get_cfg()
cfg.merge_from_file("/content/drive/MyDrive/Dataset/SafetyEquipment/config.yml")  # Update path if needed
cfg.MODEL.WEIGHTS = "/content/drive/MyDrive/Dataset/SafetyEquipment/model_final.pth"  # Trained model path
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # Confidence threshold
cfg.MODEL.DEVICE = "cuda" if torch.cuda.is_available() else "cpu"  # Use GPU if available

predictor = DefaultPredictor(cfg)

# Step 3: Upload Image
uploaded = files.upload()
image_path = list(uploaded.keys())[0]
print(f"Uploaded image: {image_path}")

# Step 4: Load Image and Run Inference
image = cv2.imread(image_path)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
outputs = predictor(image)

# Step 5: Visualize Results
v = Visualizer(image, MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
out = v.draw_instance_predictions(outputs["instances"].to("cpu"))

# Step 6: Display Image with Predictions
plt.figure(figsize=(10, 10))
plt.imshow(out.get_image())
plt.axis("off")
plt.show()

# Step 7: Save the Output (Optional)
output_filename = "output.jpg"
cv2.imwrite(output_filename, cv2.cvtColor(out.get_image(), cv2.COLOR_RGB2BGR))
print(f"Result saved as {output_filename}")


In [ ]:
# Install Detectron2
!pip install pyyaml==5.1
!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'

# Step 1: Load Dependencies
import torch
import cv2
import matplotlib.pyplot as plt
from google.colab import files
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog

# Step 2: Load Trained Model
cfg = get_cfg()
cfg.merge_from_file("/content/drive/MyDrive/Dataset/SafetyEquipment/config.yml")  # Update path if needed
cfg.MODEL.WEIGHTS = "/content/drive/MyDrive/Dataset/SafetyEquipment/model_final.pth"  # Trained model path
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # Confidence threshold
cfg.MODEL.DEVICE = "cuda" if torch.cuda.is_available() else "cpu"  # Use GPU if available

predictor = DefaultPredictor(cfg)

# Step 3: Upload Image
uploaded = files.upload()
image_path = list(uploaded.keys())[0]
print(f"Uploaded image: {image_path}")

# Step 4: Load Image and Run Inference
image = cv2.imread(image_path)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
outputs = predictor(image)

# Step 5: Visualize Results
v = Visualizer(image, MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
out = v.draw_instance_predictions(outputs["instances"].to("cpu"))

# Step 6: Display Image with Predictions
plt.figure(figsize=(10, 10))
plt.imshow(out.get_image())
plt.axis("off")
plt.show()

# Step 7: Save the Output (Optional)
output_filename = "output.jpg"
cv2.imwrite(output_filename, cv2.cvtColor(out.get_image(), cv2.COLOR_RGB2BGR))
print(f"Result saved as {output_filename}")


# TESTING **CODE**

# **FOR IMAGES segmantation**

In [ ]:
# Install Detectron2
!pip install pyyaml==5.1
!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'

# Import Required Libraries
import torch
import cv2
import matplotlib.pyplot as plt
from google.colab import files
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog
from detectron2 import model_zoo

# Load Trained Model Configuration
cfg = get_cfg()
cfg.merge_from_file("/content/drive/MyDrive/Dataset/SafetyEquipment/config.yml")
cfg.MODEL.WEIGHTS = "/content/drive/MyDrive/Dataset/SafetyEquipment/model_final.pth"
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5
cfg.MODEL.DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# Initialize Predictor
predictor = DefaultPredictor(cfg)

# Define Class Names
class_names = [
    'construction', 'Excavator', 'Gloves', 'Hardhat', 'Ladder', 'Mask', 'NO-Hardhat',
    'NO-Mask', 'NO-Safety Vest', 'Person', 'SUV', 'Safety Cone', 'Safety Vest', 'bus',
    'dump truck', 'fire hydrant', 'machinery', 'mini-van', 'sedan', 'semi', 'trailer',
    'truck', 'truck and trailer', 'van', 'vehicle', 'wheel loader'
]

# Update Metadata with Class Names
metadata = MetadataCatalog.get("safety_train")
metadata.thing_classes = class_names

# Upload Image
uploaded = files.upload()
image_path = list(uploaded.keys())[0]
print(f"Uploaded image: {image_path}")

# Load Image and Run Inference
image = cv2.imread(image_path)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
outputs = predictor(image)

# Visualize Results with Class Names
v = Visualizer(image, metadata, scale=1.2)
out = v.draw_instance_predictions(outputs["instances"].to("cpu"))

# Display Image with Predictions
plt.figure(figsize=(10, 10))
plt.imshow(out.get_image())
plt.axis("off")
plt.show()

# Save the Output (Optional)
output_filename = "output.jpg"
cv2.imwrite(output_filename, cv2.cvtColor(out.get_image(), cv2.COLOR_RGB2BGR))
print(f"Result saved as {output_filename}")


In [ ]:
import cv2
import torch
import os
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog

# Load Detectron2 Config and Model
cfg = get_cfg()
cfg.merge_from_file("/content/drive/MyDrive/Dataset/SafetyEquipment/config.yml")
cfg.MODEL.WEIGHTS = "/content/drive/MyDrive/Dataset/SafetyEquipment/model_final.pth"
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5
cfg.MODEL.DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# Initialize Predictor
predictor = DefaultPredictor(cfg)

# Define Class Names
class_names = [
    'construction', 'Excavator', 'Gloves', 'Hardhat', 'Ladder', 'Mask', 'NO-Hardhat',
    'NO-Mask', 'NO-Safety Vest', 'Person', 'SUV', 'Safety Cone', 'Safety Vest', 'bus',
    'dump truck', 'fire hydrant', 'machinery', 'mini-van', 'sedan', 'semi', 'trailer',
    'truck', 'truck and trailer', 'van', 'vehicle', 'wheel loader'
]

# Update Metadata with Class Names
metadata = MetadataCatalog.get("safety_train")
metadata.thing_classes = class_names

# Load Video
video_path = "/content/drive/MyDrive/Dataset/SafetyEquipment/Construction2.mp4"
cap = cv2.VideoCapture(video_path)

# Get Video Properties
fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

print(f"Video Loaded: {video_path}")
print(f"FPS: {fps}, Width: {width}, Height: {height}")


In [ ]:
# Define output video path
output_path = "/content/drive/MyDrive/Dataset/SafetyEquipment/CONSTRUCTIONVIDEO_DETECTED2.mp4"

# Initialize Video Writer
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec for .mp4
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

# Process Video Frame by Frame
frame_count = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break  # Stop when video ends

    # Convert BGR to RGB
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Run Object Detection
    outputs = predictor(frame_rgb)

    # Visualize Detection Results
    v = Visualizer(frame_rgb, metadata, scale=1.2)
    out_frame = v.draw_instance_predictions(outputs["instances"].to("cpu")).get_image()

    # Convert RGB to BGR (for OpenCV)
    out_frame_bgr = cv2.cvtColor(out_frame, cv2.COLOR_RGB2BGR)

    # Write Frame to Output Video
    out.write(out_frame_bgr)

    frame_count += 1
    if frame_count % 10 == 0:  # Print progress every 10 frames
        print(f"Processed {frame_count} frames...")

# Release Resources
cap.release()
out.release()
print(f"Processing complete! Video saved at: {output_path}")


In [ ]:
import cv2
import torch
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog
import matplotlib.pyplot as plt

# Load Detectron2 Config and Model
cfg = get_cfg()
cfg.merge_from_file("/content/drive/MyDrive/Dataset/SafetyEquipment/config.yml")
cfg.MODEL.WEIGHTS = "/content/drive/MyDrive/Dataset/SafetyEquipment/model_final.pth"
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5
cfg.MODEL.DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# Initialize Predictor
predictor = DefaultPredictor(cfg)

# Define Class Names
class_names = [
    'construction', 'Excavator', 'Gloves', 'Hardhat', 'Ladder', 'Mask', 'NO-Hardhat',
    'NO-Mask', 'NO-Safety Vest', 'Person', 'SUV', 'Safety Cone', 'Safety Vest', 'bus',
    'dump truck', 'fire hydrant', 'machinery', 'mini-van', 'sedan', 'semi', 'trailer',
    'truck', 'truck and trailer', 'van', 'vehicle', 'wheel loader'
]

# Update Metadata with Class Names
metadata = MetadataCatalog.get("safety_train")
metadata.thing_classes = class_names

# Load Video
video_path = "/content/drive/MyDrive/Dataset/SafetyEquipment/CONSTRUCTIONVIDEO.mp4"
cap = cv2.VideoCapture(video_path)

# Get Video Properties
fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Define Output Video Path
output_path = "/content/drive/MyDrive/Dataset/SafetyEquipment/FINAL_OUTPUT.mp4"

# Initialize Video Writer
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec for .mp4
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

# Process Video Frame by Frame
frame_count = 0
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
print(f"Total Frames: {total_frames}, FPS: {fps}, Resolution: {width}x{height}")

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break  # Stop when video ends

    # Convert BGR to RGB for Detectron2
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Run Object Detection
    outputs = predictor(frame_rgb)

    # Visualize Detection Results
    v = Visualizer(frame_rgb, metadata, scale=1.2)
    out_frame = v.draw_instance_predictions(outputs["instances"].to("cpu")).get_image()

    # Convert RGB to BGR for OpenCV
    out_frame_bgr = cv2.cvtColor(out_frame, cv2.COLOR_RGB2BGR)

    # Resize to ensure consistent frame size
    out_frame_bgr = cv2.resize(out_frame_bgr, (width, height))

    # Write Processed Frame to Output Video
    out.write(out_frame_bgr)

    frame_count += 1
    if frame_count % 10 == 0:  # Print progress every 10 frames
        print(f"Processed {frame_count}/{total_frames} frames...")

# Release Resources
cap.release()
out.release()

print(f"Processing complete! Video saved at: {output_path}")


# **For Object Tracking in Video**


In [ ]:
!pip uninstall -y opencv-python opencv-contrib-python numpy
!pip install numpy==1.23.5 opencv-contrib-python==4.5.5.64


In [ ]:


import torch
import cv2
import numpy as np
import matplotlib.pyplot as plt
from google.colab import files
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog
from google.colab.patches import cv2_imshow

# Load Trained Model Configuration
cfg = get_cfg()
cfg.merge_from_file("/content/drive/MyDrive/Dataset/SafetyEquipment/config.yml")
cfg.MODEL.WEIGHTS = "/content/drive/MyDrive/Dataset/SafetyEquipment/model_final.pth"
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5
cfg.MODEL.DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# Initialize Predictor
predictor = DefaultPredictor(cfg)

# Define Class Names
class_names = [
    'construction', 'Excavator', 'Gloves', 'Hardhat', 'Ladder', 'Mask', 'NO-Hardhat',
    'NO-Mask', 'NO-Safety Vest', 'Person', 'SUV', 'Safety Cone', 'Safety Vest', 'bus',
    'dump truck', 'fire hydrant', 'machinery', 'mini-van', 'sedan', 'semi', 'trailer',
    'truck', 'truck and trailer', 'van', 'vehicle', 'wheel loader'
]

# Update Metadata with Class Names
metadata = MetadataCatalog.get("safety_train")
metadata.thing_classes = class_names

# Upload Video
uploaded = files.upload()
video_path = list(uploaded.keys())[0]
print(f"Uploaded video: {video_path}")

# Open Video Capture
cap = cv2.VideoCapture(video_path)
tracker = cv2.TrackerCSRT_create()  # Corrected tracker creation

# Get Video Writer to save output
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
fps = int(cap.get(cv2.CAP_PROP_FPS))
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter("output.mp4", fourcc, fps, (frame_width, frame_height))

tracking_initialized = False

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    if not tracking_initialized:
        outputs = predictor(frame_rgb)
        instances = outputs["instances"].to("cpu")

        if len(instances) > 0:
            bbox = instances.pred_boxes.tensor[0].numpy()
            x, y, w, h = int(bbox[0]), int(bbox[1]), int(bbox[2] - bbox[0]), int(bbox[3] - bbox[1])
            tracker.init(frame, (x, y, w, h))
            tracking_initialized = True
    else:
        success, bbox = tracker.update(frame)
        if success:
            x, y, w, h = [int(v) for v in bbox]
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
        else:
            tracking_initialized = False  # Reinitialize if tracking fails

    cv2_imshow(frame)  # Display frame in Colab
    out.write(frame)  # Save to output video

cap.release()
out.release()
cv2.destroyAllWindows()

print("Tracking complete. Output saved as 'output.mp4'.")


In [ ]:
!pip install pyyaml==5.1
!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'
!pip install deep-sort-realtime
!pip install opencv-python
!pip install ffmpeg

In [ ]:
import torch
import cv2
import numpy as np
import matplotlib.pyplot as plt
import ffmpeg
from google.colab import files
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog
from deep_sort_realtime.deepsort_tracker import DeepSort

# Load Trained Model Configuration
cfg = get_cfg()
cfg.merge_from_file("/content/drive/MyDrive/Dataset/SafetyEquipment/config.yml")
cfg.MODEL.WEIGHTS = "/content/drive/MyDrive/Dataset/SafetyEquipment/model_final.pth"
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5
cfg.MODEL.DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# Initialize Predictor
predictor = DefaultPredictor(cfg)

# Define Class Names
class_names = [
    'construction', 'Excavator', 'Gloves', 'Hardhat', 'Ladder', 'Mask', 'NO-Hardhat',
    'NO-Mask', 'NO-Safety Vest', 'Person', 'SUV', 'Safety Cone', 'Safety Vest', 'bus',
    'dump truck', 'fire hydrant', 'machinery', 'mini-van', 'sedan', 'semi', 'trailer',
    'truck', 'truck and trailer', 'van', 'vehicle', 'wheel loader'
]

# Update Metadata with Class Names
metadata = MetadataCatalog.get("safety_train")
metadata.thing_classes = class_names

# Upload Video
uploaded = files.upload()
video_path = list(uploaded.keys())[0]
print(f"Uploaded video: {video_path}")

# Open Video Capture
cap = cv2.VideoCapture(video_path)

# Get Video Properties
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
fps = int(cap.get(cv2.CAP_PROP_FPS))

# Output Video
output_path = "veryFinalOutput.mp4"
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

# Initialize DeepSORT Tracker
tracker = DeepSort(max_age=50, n_init=3, nms_max_overlap=1.0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Object Detection
    outputs = predictor(frame_rgb)
    instances = outputs["instances"].to("cpu")

    if len(instances) > 0:
        bboxes = instances.pred_boxes.tensor.numpy()
        scores = instances.scores.numpy()
        class_ids = instances.pred_classes.numpy()

        # Format required for DeepSORT
        detection_list = []
        for i in range(len(bboxes)):
            x1, y1, x2, y2 = map(int, bboxes[i])
            detection_list.append(([x1, y1, x2, y2], scores[i], class_ids[i]))

        # Update DeepSORT tracker
        tracks = tracker.update_tracks(detection_list, frame=frame)

        for track in tracks:
            if track.is_confirmed():
                track_id = track.track_id
                x1, y1, x2, y2 = map(int, track.to_tlbr())
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(frame, f'ID {track_id}', (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

    from google.colab.patches import cv2_imshow
    cv2_imshow(frame)  # Display in Colab
    out.write(frame)

cap.release()
out.release()
cv2.destroyAllWindows()
print("Tracking complete. Output saved as 'output.mp4'.")

from google.colab import output
output.clear()


In [ ]:
from google.colab import output
output.clear()
